# 🧠 니켈 가격 예측 - 딥러닝 심화 실험

## 1. 배경 및 목적

### 1.1 sparta2 딥러닝 실험 결과
| 모델 | Val RMSE | Test RMSE | 결과 |
|------|----------|-----------|------|
| LSTM | ~461 | ~1957 | **극심한 과적합** |
| Transformer | 유사 | 유사 | **극심한 과적합** |

### 1.2 실패 원인 분석
- Train/Val 기간: 평균회귀 패턴 → DL이 완벽히 학습
- Test 기간: 일방적 급등 → 학습한 패턴과 정반대
- **결론**: DL 단독은 시장 구조 변화에 취약

### 1.3 핵심 질문
> **"DL을 ML/Naive와 결합하면 과적합을 완화하고 성능 개선 가능한가?"**

### 1.4 가설
| 가설 | 설명 | 검증 방법 |
|------|------|----------|
| H1 | DL 과적합 방지 설계로 단독 성능 개선 가능 | Dropout, 조기종료, 정규화 강화 |
| H2 | Naive + DL 앙상블로 과적합 완화 가능 | 가중 평균 (Naive 중심) |
| H3 | ML + DL 앙상블이 ML 단독보다 효과적 | GB + LSTM/Transformer 결합 |
| H4 | Quantile Regression으로 예측 불확실성 정량화 가능 | 예측 구간 커버리지 확인 |

### 1.5 실험 설계
```
Step 0: 기준선 확인 (Naive, ML, Hybrid)
    ↓
Step 1: [H1] DL 과적합 방지 설계 적용
    ↓
Step 2: [H2] Naive + DL 앙상블
    ↓
Step 3: [H3] ML + DL 앙상블
    ↓
Step 4: [H4] Quantile Regression
    ↓
결론: DL 활용 가치 평가
```

In [ ]:
# 환경 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import QuantileRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.filterwarnings('ignore')

import os
OUTPUT_DIR = 'output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CONFIG = {
    'data_file': 'data_weekly_260120.csv',
    'target_col': 'Com_LME_Ni_Cash',
    'val_start': '2025-08-04',
    'val_end': '2025-10-20',
    'test_start': '2025-10-27',
    'test_end': '2026-01-12',
    'top_n_features': 20,
    'exclude_lme_index': True,
    'random_seed': 42,
    'seq_len': 12,
}

def eval_metrics(y_true, y_pred, name=''):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
    r2 = r2_score(y_true, y_pred)
    return {'Model': name, 'RMSE': rmse, 'MAPE': mape, 'R2': r2}

np.random.seed(CONFIG['random_seed'])
torch.manual_seed(CONFIG['random_seed'])

print('='*60)
print('🧠 딥러닝 심화 실험')
print(f'Device: {device}')
print('='*60)

---
## 2. 데이터 준비 (sparta2 동일)

In [ ]:
# 데이터 로드 및 전처리 (sparta2 동일)
df_raw = pd.read_csv(CONFIG['data_file'])
df_raw['dt'] = pd.to_datetime(df_raw['dt'])
df_raw = df_raw.set_index('dt').sort_index()

def filter_cols(columns):
    target = CONFIG['target_col']
    metals = ['Gold', 'Silver', 'Iron', 'Steel', 'Copper', 'Aluminum', 'Zinc', 'Nickel', 'Lead', 'Tin', 'Uranium']
    filtered = [target]
    for col in columns:
        if any(x in col for x in ['Idx_', 'Bonds_', 'EX_']):
            filtered.append(col)
        elif 'Com_LME' in col:
            filtered.append(col)
        elif any(m in col for m in metals):
            filtered.append(col)
    return sorted(list(set(filtered)))

df = df_raw[filter_cols(df_raw.columns)].copy().ffill().dropna()
y = df[CONFIG['target_col']]
X = df.drop(columns=[CONFIG['target_col']]).shift(1)
valid_idx = X.dropna().index.intersection(y.dropna().index)
X, y = X.loc[valid_idx], y.loc[valid_idx]

if CONFIG['exclude_lme_index']:
    X = X.drop(columns=[c for c in X.columns if 'Com_LME_Index' in c], errors='ignore')

train_mask = X.index < CONFIG['val_start']
val_mask = (X.index >= CONFIG['val_start']) & (X.index <= CONFIG['val_end'])
test_mask = (X.index >= CONFIG['test_start']) & (X.index <= CONFIG['test_end'])

X_train_all, y_train = X[train_mask], y[train_mask]
X_val_all, y_val = X[val_mask], y[val_mask]
X_test_all, y_test = X[test_mask], y[test_mask]

# SHAP 피처 선택
model_shap = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_shap.fit(X_train_all, y_train)
shap_values = shap.TreeExplainer(model_shap).shap_values(X_train_all)
feat_imp = pd.DataFrame({'feature': X_train_all.columns, 'importance': np.abs(shap_values).mean(axis=0)})
selected_features = feat_imp.sort_values('importance', ascending=False).head(CONFIG['top_n_features'])['feature'].tolist()

X_train, X_val, X_test = X_train_all[selected_features], X_val_all[selected_features], X_test_all[selected_features]

print(f'✅ 데이터 준비 완료')
print(f'   Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}')

In [ ]:
# 시퀀스 데이터 준비 (DL용)
SEQ_LEN = CONFIG['seq_len']

def create_sequences(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len])
    return np.array(X_seq), np.array(y_seq)

scaler_X, scaler_y = StandardScaler(), StandardScaler()
X_all = pd.concat([X_train, X_val, X_test])
y_all = pd.concat([y_train, y_val, y_test])

X_scaled = scaler_X.fit_transform(X_all)
y_scaled = scaler_y.fit_transform(y_all.values.reshape(-1, 1)).flatten()

X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LEN)

n_train = len(X_train) - SEQ_LEN
n_val = len(X_val)
n_test = len(X_test)

X_train_seq, y_train_seq = X_seq[:n_train], y_seq[:n_train]
X_val_seq, y_val_seq = X_seq[n_train:n_train+n_val], y_seq[n_train:n_train+n_val]
X_test_seq, y_test_seq = X_seq[n_train+n_val:n_train+n_val+n_test], y_seq[n_train+n_val:n_train+n_val+n_test]

n_features = X_train_seq.shape[2]
print(f'   Seq Train: {len(X_train_seq)} | Val: {len(X_val_seq)} | Test: {len(X_test_seq)}')

---
## 3. Step 0: 기준선 확인

In [ ]:
# 기준선 모델들
prev_test = y.shift(1).loc[y_test.index].values
prev2_test = y.shift(2).loc[y_test.index].values
naive_drift_test = prev_test + (prev_test - prev2_test)

gb = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42)
gb.fit(X_train, y_train)
gb_pred_test = gb.predict(X_test)

hybrid_test = 0.8 * naive_drift_test + 0.2 * gb_pred_test

baseline_results = [
    eval_metrics(y_test, naive_drift_test, 'Naive_Drift'),
    eval_metrics(y_test, gb_pred_test, 'GradientBoosting'),
    eval_metrics(y_test, hybrid_test, 'Hybrid(0.8+0.2) [Best]'),
]
baseline_df = pd.DataFrame(baseline_results).sort_values('RMSE')

print('📊 Step 0: 기준선 (Test 기간)')
print('='*50)
display(baseline_df.round(2))

BEST_BASELINE_RMSE = baseline_df.iloc[0]['RMSE']
print(f'\n🎯 목표: RMSE < {BEST_BASELINE_RMSE:.2f}')

---
## 4. Step 1: [H1] DL 과적합 방지 설계

**가설**: 강한 정규화로 DL 단독 성능 개선 가능  
**근거**: 기존 DL은 Train 패턴에 과적합  
**방법**: Dropout 강화, 모델 축소, 조기 종료

In [ ]:
# 4.1 모델 정의 (강한 정규화 버전)
class LSTMRegularized(nn.Module):
    def __init__(self, n_feat, hidden=32, layers=1, dropout=0.3):  # 축소된 구조
        super().__init__()
        self.lstm = nn.LSTM(n_feat, hidden, layers, batch_first=True, dropout=0 if layers==1 else dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden, 1)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(self.dropout(out[:, -1])).squeeze(-1)

class TransformerRegularized(nn.Module):
    def __init__(self, n_feat, d_model=32, nhead=2, layers=1, dropout=0.3):
        super().__init__()
        self.proj = nn.Linear(n_feat, d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=64, dropout=dropout, batch_first=True)
        self.enc = nn.TransformerEncoder(enc_layer, layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, 1)
    
    def forward(self, x):
        x = self.proj(x)
        x = self.enc(x)
        return self.fc(self.dropout(x[:, -1])).squeeze(-1)

In [ ]:
# 4.2 학습 함수
def train_dl(model, X_tr, y_tr, X_vl, y_vl, epochs=80, lr=0.001, patience=15):
    model = model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)  # L2 정규화
    loss_fn = nn.MSELoss()
    
    train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_tr), torch.FloatTensor(y_tr)), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_vl), torch.FloatTensor(y_vl)), batch_size=32)
    
    best_loss, best_state, wait = np.inf, None, 0
    
    for ep in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss_fn(model(xb), yb).backward()
            opt.step()
        
        model.eval()
        val_loss = sum(loss_fn(model(xb.to(device)), yb.to(device)).item() for xb, yb in val_loader) / len(val_loader)
        
        if val_loss < best_loss:
            best_loss, best_state, wait = val_loss, {k: v.cpu().clone() for k, v in model.state_dict().items()}, 0
        else:
            wait += 1
            if wait >= patience: break
    
    if best_state: model.load_state_dict(best_state)
    return model

def predict_dl(model, X):
    model.eval()
    with torch.no_grad():
        pred = model(torch.FloatTensor(X).to(device)).cpu().numpy()
    return scaler_y.inverse_transform(pred.reshape(-1, 1)).flatten()

In [ ]:
# 4.3 LSTM & Transformer 학습
print('🔍 H1: DL 과적합 방지 설계')
print('-'*40)

print('  LSTM 학습 (정규화 강화)...')
lstm = train_dl(LSTMRegularized(n_features), X_train_seq, y_train_seq, X_val_seq, y_val_seq)
lstm_pred = predict_dl(lstm, X_test_seq)

print('  Transformer 학습 (정규화 강화)...')
transformer = train_dl(TransformerRegularized(n_features), X_train_seq, y_train_seq, X_val_seq, y_val_seq)
trans_pred = predict_dl(transformer, X_test_seq)

y_test_raw = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

h1_lstm = eval_metrics(y_test_raw, lstm_pred, 'LSTM (Regularized)')
h1_trans = eval_metrics(y_test_raw, trans_pred, 'Transformer (Regularized)')

print(f'\n📊 H1 결과:')
print(f'  LSTM: RMSE={h1_lstm["RMSE"]:.2f}')
print(f'  Transformer: RMSE={h1_trans["RMSE"]:.2f}')
print(f'  (기존 sparta2 LSTM: ~1957)')

H1_IMPROVEMENT = min(h1_lstm['RMSE'], h1_trans['RMSE']) < 1500

---
## 5. Step 2: [H2] Naive + DL 앙상블

**가설**: Naive의 추세 추종력이 DL 과적합을 보완 가능  
**근거**: Naive(0.8) + ML(0.2)가 성공적이었음  
**방법**: Naive(0.8) + DL(0.2) 구성

In [ ]:
# 5.1 길이 맞추기 (시퀀스로 줄어든 만큼)
test_len = min(len(lstm_pred), len(naive_drift_test))
naive_aligned = naive_drift_test[-test_len:]
lstm_aligned = lstm_pred[-test_len:]
trans_aligned = trans_pred[-test_len:]
y_test_aligned = y_test.values[-test_len:]

print('🔍 H2: Naive + DL 앙상블')
print('-'*40)

# Naive 0.8 + DL 0.2
naive_lstm = 0.8 * naive_aligned + 0.2 * lstm_aligned
naive_trans = 0.8 * naive_aligned + 0.2 * trans_aligned
naive_dl_avg = 0.8 * naive_aligned + 0.1 * lstm_aligned + 0.1 * trans_aligned

h2_results = [
    eval_metrics(y_test_aligned, naive_lstm, 'Naive+LSTM (0.8+0.2)'),
    eval_metrics(y_test_aligned, naive_trans, 'Naive+Transformer (0.8+0.2)'),
    eval_metrics(y_test_aligned, naive_dl_avg, 'Naive+DL_Avg (0.8+0.2)'),
]

h2_df = pd.DataFrame(h2_results).sort_values('RMSE')
print('\n📊 H2 결과:')
display(h2_df.round(2))

H2_BEST = h2_df.iloc[0]

---
## 6. Step 3: [H3] ML + DL 앙상블

**가설**: ML과 DL의 예측 다양성이 성능 향상에 기여  
**근거**: 서로 다른 패턴을 학습하므로 보완 가능  
**방법**: GB + LSTM/Transformer 가중 결합

In [ ]:
# 6.1 ML + DL 앙상블
print('🔍 H3: ML + DL 앙상블')
print('-'*40)

gb_aligned = gb_pred_test[-test_len:]

# 다양한 조합
ml_lstm = 0.7 * gb_aligned + 0.3 * lstm_aligned
ml_trans = 0.7 * gb_aligned + 0.3 * trans_aligned
ml_dl_avg = 0.6 * gb_aligned + 0.2 * lstm_aligned + 0.2 * trans_aligned

# Naive + ML + DL 3-way
three_way = 0.6 * naive_aligned + 0.2 * gb_aligned + 0.1 * lstm_aligned + 0.1 * trans_aligned

h3_results = [
    eval_metrics(y_test_aligned, ml_lstm, 'GB+LSTM (0.7+0.3)'),
    eval_metrics(y_test_aligned, ml_trans, 'GB+Transformer (0.7+0.3)'),
    eval_metrics(y_test_aligned, ml_dl_avg, 'GB+DL_Avg (0.6+0.4)'),
    eval_metrics(y_test_aligned, three_way, '3-Way (N0.6+ML0.2+DL0.2)'),
]

h3_df = pd.DataFrame(h3_results).sort_values('RMSE')
print('\n📊 H3 결과:')
display(h3_df.round(2))

H3_BEST = h3_df.iloc[0]

---
## 7. Step 4: [H4] Quantile Regression

**가설**: 점 예측 외 예측 불확실성 정량화가 의사결정에 유용  
**근거**: 급등 시장에서 불확실성이 큼  
**방법**: 10%, 25%, 50%, 75%, 90% 분위수 예측

In [ ]:
# 7.1 Quantile Regression
print('🔍 H4: Quantile Regression')
print('-'*40)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
qr_preds = {}

for q in quantiles:
    qr = QuantileRegressor(quantile=q, alpha=0.1, solver='highs')
    qr.fit(X_train_sc, y_train)
    qr_preds[q] = qr.predict(X_test_sc)
    print(f'  Q{int(q*100):02d} 완료')

# 커버리지 계산
coverage_80 = np.mean((y_test.values >= qr_preds[0.1]) & (y_test.values <= qr_preds[0.9])) * 100
coverage_50 = np.mean((y_test.values >= qr_preds[0.25]) & (y_test.values <= qr_preds[0.75])) * 100

median_rmse = np.sqrt(mean_squared_error(y_test, qr_preds[0.5]))

print(f'\n📊 H4 결과:')
print(f'  Median (Q50) RMSE: {median_rmse:.2f}')
print(f'  80% 구간 (Q10-Q90) 커버리지: {coverage_80:.1f}% (목표: 80%)')
print(f'  50% 구간 (Q25-Q75) 커버리지: {coverage_50:.1f}% (목표: 50%)')

H4_VALID = coverage_80 >= 70  # 80%에 근접하면 유효

In [ ]:
# 7.2 예측 구간 시각화
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(y_test.index, y_test.values, 'k-', label='Actual', lw=2)
ax.plot(y_test.index, qr_preds[0.5], 'b--', label='Median (Q50)', lw=1.5)
ax.fill_between(y_test.index, qr_preds[0.1], qr_preds[0.9], alpha=0.2, color='blue', label='80% PI')
ax.fill_between(y_test.index, qr_preds[0.25], qr_preds[0.75], alpha=0.3, color='blue', label='50% PI')

ax.set_title(f'Quantile Regression (80% Coverage: {coverage_80:.1f}%)', fontweight='bold')
ax.set_ylabel('Nickel Price (USD/t)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/quantile_regression.png', dpi=150)
plt.show()

---
## 8. 최종 결과 및 결론

In [ ]:
# 8.1 전체 결과 통합
all_results = baseline_results + [h1_lstm, h1_trans] + h2_results + h3_results
all_results.append(eval_metrics(y_test, qr_preds[0.5], 'QR Median'))

final_df = pd.DataFrame(all_results).sort_values('RMSE')

print('='*70)
print('📊 전체 결과 (Test 기간)')
print('='*70)
display(final_df[['Model', 'RMSE', 'MAPE', 'R2']].round(2))

In [ ]:
# 8.2 가설 검증 결과
print('='*70)
print('📋 가설 검증 결과')
print('='*70)

print(f'\n[H1] DL 과적합 방지 설계')
print(f'     결과: {"✅ 효과 있음" if H1_IMPROVEMENT else "❌ 여전히 과적합"}')
print(f'     LSTM: {h1_lstm["RMSE"]:.2f}, Transformer: {h1_trans["RMSE"]:.2f}')
print(f'     (기존 sparta2 LSTM: ~1957)')

print(f'\n[H2] Naive + DL 앙상블')
naive_dl_improved = H2_BEST['RMSE'] < min(h1_lstm['RMSE'], h1_trans['RMSE'])
print(f'     결과: {"✅ 개선" if naive_dl_improved else "❌ 개선 없음"}')
print(f'     Best: {H2_BEST["Model"]} (RMSE={H2_BEST["RMSE"]:.2f})')

print(f'\n[H3] ML + DL 앙상블')
ml_dl_improved = H3_BEST['RMSE'] < baseline_df[baseline_df['Model']=='GradientBoosting']['RMSE'].values[0]
print(f'     결과: {"✅ GB 대비 개선" if ml_dl_improved else "❌ GB 대비 열등"}')
print(f'     Best: {H3_BEST["Model"]} (RMSE={H3_BEST["RMSE"]:.2f})')

print(f'\n[H4] Quantile Regression')
print(f'     결과: {"✅ 유효" if H4_VALID else "⚠️ 커버리지 부족"}')
print(f'     80% 구간 커버리지: {coverage_80:.1f}%')

In [ ]:
# 8.3 최종 결론
best = final_df.iloc[0]
improvement = (BEST_BASELINE_RMSE - best['RMSE']) / BEST_BASELINE_RMSE * 100

print('\n' + '='*70)
print('🏆 최종 결론')
print('='*70)

print(f'\n🎯 최고 성능 모델: {best["Model"]}')
print(f'   RMSE: {best["RMSE"]:.2f}')
print(f'   기준선 대비: {improvement:+.2f}%')

print(f'\n📌 DL 활용 가치 평가:')
if best['RMSE'] < BEST_BASELINE_RMSE:
    print(f'   ✅ DL 기반 앙상블이 기준선 개선에 기여')
else:
    print(f'   ⚠️ DL 단독/앙상블이 Naive+ML Hybrid를 넘지 못함')
    print(f'   → DL은 시장 구조 변화에 여전히 취약')
    print(f'   → 추가 가치: Quantile Regression으로 불확실성 정량화')

print(f'\n📌 실무 권장:')
print(f'   1순위: Hybrid(Naive+ML) - 안정적이고 해석 가능')
print(f'   2순위: Quantile Regression - 예측 구간 제공')
print(f'   DL: 보조 앙상블 멤버로만 활용')
print('='*70)